In [18]:
# %% [cell 1] - Flask API Code
from flask import Flask, request, jsonify
from flask_cors import CORS
import bcrypt
import requests
import json
from datetime import datetime, timedelta
import paho.mqtt.publish as publish
from dotenv import load_dotenv
import os
import threading
import time

# تحميل متغيرات البيئة
load_dotenv()

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}}, supports_credentials=True)

# محاكاة Supabase (للتجربة)
class MockSupabase:
    def __init__(self):
        self.users = []
        self.sensor_readings = []
        self.alerts = []
        self.device_status = []
        self.access_logs = []
        self.notifications = []
        self.chat_messages = []
    
    def table(self, table_name):
        return MockTable(getattr(self, table_name))

class MockTable:
    def __init__(self, data):
        self.data = data
    
    def select(self, *args):
        return self
    
    def insert(self, item):
        item['id'] = len(self.data) + 1
        self.data.append(item)
        return self
    
    def update(self, item):
        return self
    
    def delete(self):
        return self
    
    def eq(self, key, value):
        filtered_data = [item for item in self.data if item.get(key) == value]
        return MockTable(filtered_data)
    
    def execute(self):
        return MockResponse(self.data)
    
    def order(self, key, desc=True):
        sorted_data = sorted(self.data, key=lambda x: x.get(key, ''), reverse=desc)
        return MockTable(sorted_data)
    
    def limit(self, limit):
        limited_data = self.data[:limit]
        return MockTable(limited_data)

class MockResponse:
    def __init__(self, data):
        self.data = data

# تهيئة Mock Supabase
supabase = MockSupabase()

# ========== MQTT Helper ==========
def mqtt_publish(topic: str, payload: str, qos: int = 0, retain: bool = False):
    print(f"📤 MQTT Publish: {topic} -> {payload}")
    # محاكاة MQTT للاختبار
    pass

# ========== Authentication Routes ==========
@app.post("/auth/signup")
def auth_signup():
    data = request.get_json() or {}
    email = data.get("email")
    password = data.get("password")
    name = data.get("name", "")
    
    if not email or not password:
        return jsonify({"error": "البريد الإلكتروني وكلمة المرور مطلوبان"}), 400

    try:
        # محاكاة إنشاء مستخدم
        user = {
            "id": str(len(supabase.users) + 1),
            "email": email,
            "name": name,
            "password": password  # في الواقع يجب تشفيرها
        }
        
        supabase.users.append(user)
        
        return jsonify({
            "id": user["id"], 
            "email": email, 
            "name": name,
            "message": "تم إنشاء الحساب بنجاح"
        }), 201
        
    except Exception as e:
        return jsonify({"error": f"خطأ في إنشاء الحساب: {str(e)}"}), 400

@app.post("/auth/login")
def auth_login():
    data = request.get_json() or {}
    email = data.get("email")
    password = data.get("password")
    
    if not email or not password:
        return jsonify({"error": "البريد الإلكتروني وكلمة المرور مطلوبان"}), 400

    try:
        # محاكاة تسجيل الدخول
        user = next((u for u in supabase.users if u["email"] == email and u["password"] == password), None)
        
        if user:
            return jsonify({
                "access_token": "mock_token_" + user["id"],
                "refresh_token": "mock_refresh_token_" + user["id"],
                "user": {
                    "id": user["id"], 
                    "email": user["email"],
                    "name": user["name"]
                },
                "message": "تم تسجيل الدخول بنجاح"
            })
        else:
            return jsonify({"error": "بيانات الدخول غير صحيحة"}), 401
        
    except Exception as e:
        return jsonify({"error": "بيانات الدخول غير صحيحة", "details": str(e)}), 401

# ========== Dashboard Routes ==========
@app.get("/dashboard/data")
def dashboard_data():
    try:
        # بيانات تجريبية
        sensor_data = {
            "temperature": 24.5,
            "humidity": 65,
            "gas": 250,
            "current": 12.5,
            "motion": 0,
            "door_status": "closed",
            "light_status": "off",
            "timestamp": datetime.now().isoformat()
        }
        
        alerts_data = [
            {
                "id": 1,
                "alert_type": "info",
                "message": "System started successfully",
                "timestamp": datetime.now().isoformat(),
                "resolved": True
            }
        ]
        
        devices_data = [
            {
                "device_type": "door",
                "status": "closed",
                "last_updated": datetime.now().isoformat()
            },
            {
                "device_type": "light",
                "status": "off",
                "last_updated": datetime.now().isoformat()
            }
        ]
        
        return jsonify({
            "sensors": sensor_data,
            "alerts": alerts_data,
            "devices": devices_data,
            "message": "تم جلب بيانات Dashboard بنجاح"
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# ========== Device Control Routes ==========
@app.post("/control/door")
def control_door():
    data = request.get_json() or {}
    action = data.get("action")
    user_id = data.get("user_id")
    
    if action not in ["open", "close"]:
        return jsonify({"error": "الإجراء غير صحيح. الاختيارات المتاحة: open, close"}), 400
        
    try:
        # إرسال أمر التحكم عن طريق MQTT
        mqtt_publish("esp32/door/control", action, qos=2, retain=False)
        
        # تسجيل العملية
        supabase.access_logs.append({
            "user_id": user_id,
            "action": f"door_{action}",
            "success": True,
            "timestamp": datetime.now().isoformat()
        })
        
        return jsonify({
            "success": True, 
            "message": f"تم {action} الباب بنجاح"
        })
        
    except Exception as e:
        return jsonify({"error": f"خطأ في التحكم بالباب: {str(e)}"}), 500

@app.post("/control/light")
def control_light():
    data = request.get_json() or {}
    action = data.get("action")
    user_id = data.get("user_id")
    device_id = data.get("device_id", "1")
    
    if action not in ["on", "off"]:
        return jsonify({"error": "الإجراء غير صحيح. الاختيارات المتاحة: on, off"}), 400
        
    try:
        # إرسال أمر التحكم عن طريق MQTT
        mqtt_publish("esp32/light/control", action, qos=2, retain=False)
        
        # تسجيل العملية
        supabase.access_logs.append({
            "user_id": user_id,
            "action": f"light_{action}",
            "device_id": device_id,
            "success": True,
            "timestamp": datetime.now().isoformat()
        })
        
        return jsonify({
            "success": True, 
            "message": f"تم {action} الضوء بنجاح"
        })
        
    except Exception as e:
        return jsonify({"error": f"خطأ في التحكم بالضوء: {str(e)}"}), 500

# ========== Alerts Routes ==========
@app.get("/alerts")
def get_alerts():
    try:
        alert_type = request.args.get("type", "all")
        limit = int(request.args.get("limit", 20))
        
        alerts_data = supabase.alerts[:limit]
        
        return jsonify({
            "data": alerts_data,
            "total": len(alerts_data),
            "message": "تم جلب الإنذارات بنجاح"
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# ========== Notifications Routes ==========
@app.get("/notifications")
def get_notifications():
    try:
        limit = int(request.args.get("limit", 50))
        
        notifications_data = supabase.notifications[:limit]
        
        return jsonify({
            "data": notifications_data,
            "total": len(notifications_data),
            "unread_count": len([n for n in notifications_data if not n.get("read")]),
            "message": "تم جلب الإشعارات بنجاح"
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# ========== History Routes ==========
@app.get("/history")
def get_history():
    try:
        limit = int(request.args.get("limit", 50))
        
        history_data = supabase.access_logs[:limit]
        
        return jsonify({
            "data": history_data,
            "total": len(history_data),
            "message": "تم جلب التاريخ بنجاح"
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# ========== Chatbot Routes ==========
@app.post("/chatbot/message")
def chatbot_message():
    try:
        data = request.get_json() or {}
        message = data.get("message")
        user_id = data.get("user_id")
        
        if not message:
            return jsonify({"error": "الرسالة مطلوبة"}), 400
        
        # حفظ رسالة المستخدم
        supabase.chat_messages.append({
            "user_id": user_id,
            "message": message,
            "sender": "user",
            "timestamp": datetime.now().isoformat()
        })
        
        # توليد رد المساعد
        response = generate_chatbot_response(message)
        
        # حفظ رد المساعد
        supabase.chat_messages.append({
            "user_id": user_id,
            "message": response,
            "sender": "bot",
            "timestamp": datetime.now().isoformat()
        })
        
        return jsonify({
            "response": response,
            "message": "تم استلام الرسالة بنجاح"
        })
        
    except Exception as e:
        return jsonify({"error": str(e)}), 500

def generate_chatbot_response(message):
    """توليد ردود ذكية للمساعد"""
    lower_message = message.lower()
    
    if any(word in lower_message for word in ['hello', 'hi', 'مرحبا', 'اهلا']):
        return "Hello! How can I assist you with your smart home today?"
    elif any(word in lower_message for word in ['door', 'باب', 'فتح', 'غلق']):
        return "I can help you with door controls. Would you like to lock/unlock the door or check the door status?"
    elif any(word in lower_message for word in ['light', 'ضوء', 'لمبة', 'نور']):
        return "I can control your lights. Which room's lights would you like to adjust?"
    elif any(word in lower_message for word in ['temperature', 'حرارة', 'طقس']):
        return "The current temperature is 24.5°C. Would you like to adjust the thermostat?"
    elif any(word in lower_message for word in ['help', 'مساعدة', 'مساعده']):
        return "I'm here to help! You can ask me about door controls, light controls, temperature settings, security alerts, and system status."
    else:
        return "I understand. How can I help you with that?"

# ========== Health Check ==========
@app.get("/health")
def health():
    return jsonify({"status": "ok", "message": "الخادم يعمل بشكل طبيعي"})

print("✅ Flask app configured successfully!")

✅ Flask app configured successfully!


In [19]:
# %% [cell 2] - Run Flask Server
import threading
import time
from werkzeug.serving import make_server

class ServerThread(threading.Thread):
    def __init__(self, app, host='0.0.0.0', port=5000):
        threading.Thread.__init__(self)
        self.srv = make_server(host, port, app)
        self.ctx = app.app_context()
        self.ctx.push()
        self.daemon = True  # يجعل الثريد يغلق عند إغلاق النوتبوك

    def run(self):
        print("🚀 Starting Flask server on http://127.0.0.1:5000")
        print("📋 Available endpoints:")
        print("   • GET  /health")
        print("   • POST /auth/signup")
        print("   • POST /auth/login")
        print("   • GET  /dashboard/data")
        print("   • POST /control/door")
        print("   • POST /control/light")
        print("   • GET  /alerts")
        print("   • GET  /notifications")
        print("   • GET  /history")
        print("   • POST /chatbot/message")
        print("\n⏹️  To stop the server: kernel -> restart")
        self.srv.serve_forever()

    def shutdown(self):
        print("🛑 Shutting down Flask server...")
        self.srv.shutdown()

# تشغيل السيرفر في ثريد منفصل
server_thread = ServerThread(app)
server_thread.start()

# حفظ reference لإمكانية إيقاف السيرفر لاحقاً
flask_server = server_thread

print("\n🎯 Server is running! You can now:")
print("   • Open http://127.0.0.1:5000/health in your browser")
print("   • Test endpoints using curl or Postman")
print("   • Use the frontend pages we created")

# إضافة بعض البيانات التجريبية للاختبار
supabase.users.append({
    "id": "1",
    "email": "test@example.com",
    "name": "Test User",
    "password": "password123"
})

supabase.alerts.append({
    "id": 1,
    "alert_type": "info",
    "message": "System started successfully",
    "timestamp": datetime.now().isoformat(),
    "resolved": True
})

supabase.notifications.append({
    "id": 1,
    "type": "system",
    "message": "Welcome to Smart Home System!",
    "read": False,
    "created_at": datetime.now().isoformat()
})

# البقاء في حلقة无限 للحفاظ على السيرفر شغال
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    flask_server.shutdown()
    print("Server stopped by user")

🚀 Starting Flask server on http://127.0.0.1:5000
📋 Available endpoints:
   • GET  /health
   • POST /auth/signup
   • POST /auth/login
   • GET  /dashboard/data
   • POST /control/door
   • POST /control/light
   • GET  /alerts
   • GET  /notifications
   • GET  /history
   • POST /chatbot/message

⏹️  To stop the server: kernel -> restart

🎯 Server is running! You can now:
   • Open http://127.0.0.1:5000/health in your browser
   • Test endpoints using curl or Postman
   • Use the frontend pages we created
🛑 Shutting down Flask server...
Server stopped by user


In [24]:
# %% [cell 1] - Flask API Code
from flask import Flask, request, jsonify
from flask_cors import CORS
import bcrypt
import requests
import json
from datetime import datetime, timedelta
import paho.mqtt.publish as publish
from dotenv import load_dotenv
import os
import threading
import time
import re  # إضافة مكتبة regex للتحقق من صحة البريد الإلكتروني

# تحميل متغيرات البيئة
load_dotenv()

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}}, supports_credentials=True)

# محاكاة Supabase (للتجربة)
class MockSupabase:
    def __init__(self):
        self.users = []
        self.sensor_readings = []
        self.alerts = []
        self.device_status = []
        self.access_logs = []
        self.notifications = []
        self.chat_messages = []
    
    def table(self, table_name):
        return MockTable(getattr(self, table_name))

class MockTable:
    def __init__(self, data):
        self.data = data
    
    def select(self, *args):
        return self
    
    def insert(self, item):
        item['id'] = len(self.data) + 1
        self.data.append(item)
        return self
    
    def update(self, item):
        return self
    
    def delete(self):
        return self
    
    def eq(self, key, value):
        filtered_data = [item for item in self.data if item.get(key) == value]
        return MockTable(filtered_data)
    
    def execute(self):
        return MockResponse(self.data)
    
    def order(self, key, desc=True):
        sorted_data = sorted(self.data, key=lambda x: x.get(key, ''), reverse=desc)
        return MockTable(sorted_data)
    
    def limit(self, limit):
        limited_data = self.data[:limit]
        return MockTable(limited_data)

class MockResponse:
    def __init__(self, data):
        self.data = data

# تهيئة Mock Supabase
supabase = MockSupabase()

# ========== MQTT Helper ==========
def mqtt_publish(topic: str, payload: str, qos: int = 0, retain: bool = False):
    print(f"📤 MQTT Publish: {topic} -> {payload}")
    # محاكاة MQTT للاختبار
    pass

# ========== Authentication Routes ==========
@app.post("/auth/signup")
def auth_signup():
    data = request.get_json() or {}
    email = data.get("email")
    password = data.get("password")
    name = data.get("name", "")
    
    if not email or not password:
        return jsonify({"error": "البريد الإلكتروني وكلمة المرور مطلوبان", "msg": "يرجى ملء جميع الحقول"}), 400

    # التحقق من صحة البريد الإلكتروني
    if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
        return jsonify({"error": "بريد إلكتروني غير صحيح", "msg": "صيغة البريد الإلكتروني غير صحيحة"}), 400
    
    # التحقق من قوة كلمة المرور
    if len(password) < 6:
        return jsonify({"error": "كلمة المرور ضعيفة", "msg": "كلمة المرور يجب أن تكون على الأقل 6 أحرف"}), 400

    try:
        # التحقق من عدم وجود مستخدم بنفس البريد الإلكتروني
        if any(user["email"] == email for user in supabase.users):
            return jsonify({"error": "البريد الإلكتروني مسجل مسبقاً", "msg": "هذا البريد الإلكتروني مستخدم بالفعل"}), 409
        
        # تشفير كلمة المرور باستخدام bcrypt
        hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
        
        # إنشاء مستخدم جديد
        user = {
            "id": str(len(supabase.users) + 1),
            "email": email,
            "name": name,
            "password": hashed_password,
            "created_at": datetime.now().isoformat()
        }
        
        supabase.users.append(user)
        
        return jsonify({
            "id": user["id"], 
            "email": email, 
            "name": name,
            "message": "تم إنشاء الحساب بنجاح",
            "msg": "تم إنشاء الحساب بنجاح"
        }), 201
        
    except Exception as e:
        return jsonify({"error": f"خطأ في إنشاء الحساب: {str(e)}", "msg": "حدث خطأ غير متوقع"}), 400

@app.post("/auth/login")
def auth_login():
    data = request.get_json() or {}
    email = data.get("email")
    password = data.get("password")
    
    if not email or not password:
        return jsonify({"error": "البريد الإلكتروني وكلمة المرور مطلوبان", "msg": "يرجى إدخال البريد الإلكتروني وكلمة المرور"}), 400

    try:
        # البحث عن المستخدم
        user = next((u for u in supabase.users if u["email"] == email), None)
        
        if user and bcrypt.checkpw(password.encode('utf-8'), user["password"].encode('utf-8')):
            return jsonify({
                "access_token": "mock_token_" + user["id"],
                "refresh_token": "mock_refresh_token_" + user["id"],
                "user": {
                    "id": user["id"], 
                    "email": user["email"],
                    "name": user["name"]
                },
                "message": "تم تسجيل الدخول بنجاح",
                "msg": "تم تسجيل الدخول بنجاح"
            })
        else:
            return jsonify({"error": "بيانات الدخول غير صحيحة", "msg": "البريد الإلكتروني أو كلمة المرور غير صحيحة"}), 401
        
    except Exception as e:
        return jsonify({"error": "بيانات الدخول غير صحيحة", "msg": "حدث خطأ أثناء تسجيل الدخول", "details": str(e)}), 401

@app.get("/auth/verify")
def auth_verify():
    token = request.headers.get('Authorization', '').replace('Bearer ', '')
    
    if not token or not token.startswith('mock_token_'):
        return jsonify({"error": "توكن غير صالح", "msg": "جلسة العمل منتهية، يرجى تسجيل الدخول مرة أخرى"}), 401
    
    user_id = token.replace('mock_token_', '')
    user = next((u for u in supabase.users if u["id"] == user_id), None)
    
    if user:
        return jsonify({
            "valid": True,
            "user": {
                "id": user["id"], 
                "email": user["email"],
                "name": user["name"]
            },
            "msg": "التوكن صالح"
        })
    else:
        return jsonify({"error": "توكن غير صالح", "msg": "جلسة العمل منتهية", "valid": False}), 401

# ========== Dashboard Routes ==========
@app.get("/dashboard/data")
def dashboard_data():
    try:
        # بيانات تجريبية
        sensor_data = {
            "temperature": 24.5,
            "humidity": 65,
            "gas": 250,
            "current": 12.5,
            "motion": 0,
            "door_status": "closed",
            "light_status": "off",
            "timestamp": datetime.now().isoformat()
        }
        
        alerts_data = [
            {
                "id": 1,
                "alert_type": "info",
                "message": "System started successfully",
                "timestamp": datetime.now().isoformat(),
                "resolved": True
            }
        ]
        
        devices_data = [
            {
                "device_type": "door",
                "status": "closed",
                "last_updated": datetime.now().isoformat()
            },
            {
                "device_type": "light",
                "status": "off",
                "last_updated": datetime.now().isoformat()
            }
        ]
        
        return jsonify({
            "sensors": sensor_data,
            "alerts": alerts_data,
            "devices": devices_data,
            "message": "تم جلب بيانات Dashboard بنجاح",
            "msg": "تم جلب البيانات بنجاح"
        })
    except Exception as e:
        return jsonify({"error": str(e), "msg": "حدث خطأ أثناء جلب البيانات"}), 500

# ========== Device Control Routes ==========
@app.post("/control/door")
def control_door():
    data = request.get_json() or {}
    action = data.get("action")
    user_id = data.get("user_id")
    
    if action not in ["open", "close"]:
        return jsonify({"error": "الإجراء غير صحيح. الاختيارات المتاحة: open, close", "msg": "إجراء غير صحيح"}), 400
        
    try:
        # إرسال أمر التحكم عن طريق MQTT
        mqtt_publish("esp32/door/control", action, qos=2, retain=False)
        
        # تسجيل العملية
        supabase.access_logs.append({
            "user_id": user_id,
            "action": f"door_{action}",
            "success": True,
            "timestamp": datetime.now().isoformat()
        })
        
        return jsonify({
            "success": True, 
            "message": f"تم {action} الباب بنجاح",
            "msg": f"تم {action} الباب بنجاح"
        })
        
    except Exception as e:
        return jsonify({"error": f"خطأ في التحكم بالباب: {str(e)}", "msg": "حدث خطأ أثناء التحكم بالباب"}), 500

@app.post("/control/light")
def control_light():
    data = request.get_json() or {}
    action = data.get("action")
    user_id = data.get("user_id")
    device_id = data.get("device_id", "1")
    
    if action not in ["on", "off"]:
        return jsonify({"error": "الإجراء غير صحيح. الاختيارات المتاحة: on, off", "msg": "إجراء غير صحيح"}), 400
        
    try:
        # إرسال أمر التحكم عن طريق MQTT
        mqtt_publish("esp32/light/control", action, qos=2, retain=False)
        
        # تسجيل العملية
        supabase.access_logs.append({
            "user_id": user_id,
            "action": f"light_{action}",
            "device_id": device_id,
            "success": True,
            "timestamp": datetime.now().isoformat()
        })
        
        return jsonify({
            "success": True, 
            "message": f"تم {action} الضوء بنجاح",
            "msg": f"تم {action} الضوء بنجاح"
        })
        
    except Exception as e:
        return jsonify({"error": f"خطأ في التحكم بالضوء: {str(e)}", "msg": "حدث خطأ أثناء التحكم بالضوء"}), 500

# ========== Alerts Routes ==========
@app.get("/alerts")
def get_alerts():
    try:
        alert_type = request.args.get("type", "all")
        limit = int(request.args.get("limit", 20))
        
        alerts_data = supabase.alerts[:limit]
        
        return jsonify({
            "data": alerts_data,
            "total": len(alerts_data),
            "message": "تم جلب الإنذارات بنجاح",
            "msg": "تم جلب الإنذارات بنجاح"
        })
    except Exception as e:
        return jsonify({"error": str(e), "msg": "حدث خطأ أثناء جلب الإنذارات"}), 500

# ========== Notifications Routes ==========
@app.get("/notifications")
def get_notifications():
    try:
        limit = int(request.args.get("limit", 50))
        
        notifications_data = supabase.notifications[:limit]
        
        return jsonify({
            "data": notifications_data,
            "total": len(notifications_data),
            "unread_count": len([n for n in notifications_data if not n.get("read")]),
            "message": "تم جلب الإشعارات بنجاح",
            "msg": "تم جلب الإشعارات بنجاح"
        })
    except Exception as e:
        return jsonify({"error": str(e), "msg": "حدث خطأ أثناء جلب الإشعارات"}), 500

# ========== History Routes ==========
@app.get("/history")
def get_history():
    try:
        limit = int(request.args.get("limit", 50))
        
        history_data = supabase.access_logs[:limit]
        
        return jsonify({
            "data": history_data,
            "total": len(history_data),
            "message": "تم جلب التاريخ بنجاح",
            "msg": "تم جلب التاريخ بنجاح"
        })
    except Exception as e:
        return jsonify({"error": str(e), "msg": "حدث خطأ أثناء جلب التاريخ"}), 500

# ========== Chatbot Routes ==========
@app.post("/chatbot/message")
def chatbot_message():
    try:
        data = request.get_json() or {}
        message = data.get("message")
        user_id = data.get("user_id")
        
        if not message:
            return jsonify({"error": "الرسالة مطلوبة", "msg": "الرسالة مطلوبة"}), 400
        
        # حفظ رسالة المستخدم
        supabase.chat_messages.append({
            "user_id": user_id,
            "message": message,
            "sender": "user",
            "timestamp": datetime.now().isoformat()
        })
        
        # توليد رد المساعد
        response = generate_chatbot_response(message)
        
        # حفظ رد المساعد
        supabase.chat_messages.append({
            "user_id": user_id,
            "message": response,
            "sender": "bot",
            "timestamp": datetime.now().isoformat()
        })
        
        return jsonify({
            "response": response,
            "message": "تم استلام الرسالة بنجاح",
            "msg": "تم استلام الرسالة بنجاح"
        })
        
    except Exception as e:
        return jsonify({"error": str(e), "msg": "حدث خطأ أثناء معالجة الرسالة"}), 500

def generate_chatbot_response(message):
    """توليد ردود ذكية للمساعد"""
    lower_message = message.lower()
    
    if any(word in lower_message for word in ['hello', 'hi', 'مرحبا', 'اهلا']):
        return "Hello! How can I assist you with your smart home today?"
    elif any(word in lower_message for word in ['door', 'باب', 'فتح', 'غلق']):
        return "I can help you with door controls. Would you like to lock/unlock the door or check the door status?"
    elif any(word in lower_message for word in ['light', 'ضوء', 'لمبة', 'نور']):
        return "I can control your lights. Which room's lights would you like to adjust?"
    elif any(word in lower_message for word in ['temperature', 'حرارة', 'طقس']):
        return "The current temperature is 24.5°C. Would you like to adjust the thermostat?"
    elif any(word in lower_message for word in ['help', 'مساعدة', 'مساعده']):
        return "I'm here to help! You can ask me about door controls, light controls, temperature settings, security alerts, and system status."
    else:
        return "I understand. How can I help you with that?"

# ========== Health Check ==========
@app.get("/health")
def health():
    return jsonify({"status": "ok", "message": "الخادم يعمل بشكل طبيعي", "msg": "الخادم يعمل بشكل طبيعي"})

print("✅ Flask app configured successfully!")

# %% [cell 2] - Run Flask Server
import threading
import time
from werkzeug.serving import make_server

class ServerThread(threading.Thread):
    def __init__(self, app, host='0.0.0.0', port=5000):
        threading.Thread.__init__(self)
        self.srv = make_server(host, port, app)
        self.ctx = app.app_context()
        self.ctx.push()
        self.daemon = True  # يجعل الثريد يغلق عند إغلاق النوتبوك

    def run(self):
        print("🚀 Starting Flask server on http://127.0.0.1:5000")
        print("📋 Available endpoints:")
        print("   • GET  /health")
        print("   • POST /auth/signup")
        print("   • POST /auth/login")
        print("   • GET  /auth/verify")
        print("   • GET  /dashboard/data")
        print("   • POST /control/door")
        print("   • POST /control/light")
        print("   • GET  /alerts")
        print("   • GET  /notifications")
        print("   • GET  /history")
        print("   • POST /chatbot/message")
        print("\n⏹️  To stop the server: kernel -> restart")
        self.srv.serve_forever()

    def shutdown(self):
        print("🛑 Shutting down Flask server...")
        self.srv.shutdown()

# تشغيل السيرفر في ثريد منفصل
server_thread = ServerThread(app)
server_thread.start()

# حفظ reference لإمكانية إيقاف السيرفر لاحقاً
flask_server = server_thread

print("\n🎯 Server is running! You can now:")
print("   • Open http://127.0.0.1:5000/health in your browser")
print("   • Test endpoints using curl or Postman")
print("   • Use the frontend pages we created")

# إضافة بعض البيانات التجريبية للاختبار
supabase.users.append({
    "id": "1",
    "email": "test@example.com",
    "name": "Test User",
    "password": bcrypt.hashpw("password123".encode('utf-8'), bcrypt.gensalt()).decode('utf-8'),
    "created_at": datetime.now().isoformat()
})

supabase.alerts.append({
    "id": 1,
    "alert_type": "info",
    "message": "System started successfully",
    "timestamp": datetime.now().isoformat(),
    "resolved": True
})

supabase.notifications.append({
    "id": 1,
    "type": "system",
    "message": "Welcome to Smart Home System!",
    "read": False,
    "created_at": datetime.now().isoformat()
})

# البقاء في حلقة无限 للحفاظ على السيرفر شغال
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    flask_server.shutdown()
    print("Server stopped by user")

✅ Flask app configured successfully!
🚀 Starting Flask server on http://127.0.0.1:5000
📋 Available endpoints:
   • GET  /health
   • POST /auth/signup
   • POST /auth/login
   • GET  /auth/verify
   • GET  /dashboard/data
   • POST /control/door
   • POST /control/light
   • GET  /alerts
   • GET  /notifications
   • GET  /history
   • POST /chatbot/message

⏹️  To stop the server: kernel -> restart

🎯 Server is running! You can now:
   • Open http://127.0.0.1:5000/health in your browser
   • Test endpoints using curl or Postman
   • Use the frontend pages we created
🛑 Shutting down Flask server...
Server stopped by user


In [ ]:
# %% [cell 1] - Flask API + MQTT Integration
from flask import Flask, request, jsonify
from flask_cors import CORS
import bcrypt
import json
from datetime import datetime
import paho.mqtt.client as mqtt
import threading

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# ====== Database Simulation ======
class DB:
    def __init__(self):
        self.users = []
        self.sensors_reading = []
        self.alerts = []
        self.mqtt_messages = []
        self.history = []
        self.access_logs = []

db = DB()

# ===== MQTT Config =====
MQTT_BROKER = "26f8d419f7fc4342bca9e0967e0dc42d.s1.eu.hivemq.cloud"
MQTT_PORT = 8883
MQTT_USER = "hivemq.webclient.1756491754332"
MQTT_PASS = "W:,3QqPG172.rFxej$uS"
MQTT_TOPICS = [("esp32/sensors/data", 1),
               ("esp32/alerts", 1),
               ("esp32/history", 1)]

# ===== MQTT Callbacks =====
def on_connect(client, userdata, flags, rc):
    print("✅ MQTT Connected, result code="+str(rc))
    client.subscribe(MQTT_TOPICS)

def on_message(client, userdata, msg):
    payload = msg.payload.decode()
    topic = msg.topic
    timestamp = datetime.now().isoformat()

    # حفظ كل رسالة MQTT
    db.mqtt_messages.append({
        "topic": topic,
        "payload": payload,
        "timestamp": timestamp
    })

    # ======= معالجة البيانات حسب الـ topic =======
    try:
        data = json.loads(payload)
    except:
        data = {}

    if topic == "esp32/sensors/data":
        for key, value in data.items():
            db.sensors_reading.append({
                "sensor_name": key,
                "value": value,
                "timestamp": timestamp
            })

    elif topic == "esp32/alerts":
        data["id"] = len(db.alerts) + 1
        data["timestamp"] = timestamp
        db.alerts.append(data)

    elif topic == "esp32/history":
        data["timestamp"] = timestamp
        db.history.append(data)

# ===== MQTT Client Thread =====
mqtt_client = mqtt.Client()
mqtt_client.username_pw_set(MQTT_USER, MQTT_PASS)
mqtt_client.tls_set()  # TLS since port 8883
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message
mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)

threading.Thread(target=mqtt_client.loop_forever, daemon=True).start()

# ================== Flask Endpoints ==================

@app.get("/sensors")
def get_sensors():
    return jsonify({"data": db.sensors_reading[-50:], "total": len(db.sensors_reading)})

@app.get("/alerts")
def get_alerts():
    return jsonify({"data": db.alerts[-50:], "total": len(db.alerts)})

@app.get("/history")
def get_history():
    return jsonify({"data": db.history[-50:], "total": len(db.history)})

@app.get("/mqtt_messages")
def get_mqtt_messages():
    return jsonify({"data": db.mqtt_messages[-50:], "total": len(db.mqtt_messages)})

# ===== Authentication Example =====
@app.post("/auth/signup")
def signup():
    data = request.get_json() or {}
    email = data.get("email")
    password = data.get("password")
    name = data.get("name", "")
    if not email or not password:
        return jsonify({"error": "Email and password required"}), 400
    if any(u["email"]==email for u in db.users):
        return jsonify({"error": "Email exists"}), 409
    hashed = bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()
    user = {"id": str(len(db.users)+1), "email": email, "name": name, "password": hashed, "created_at": datetime.now().isoformat()}
    db.users.append(user)
    return jsonify({"message":"User created","user":user})

@app.post("/auth/login")
def login():
    data = request.get_json() or {}
    email = data.get("email")
    password = data.get("password")
    user = next((u for u in db.users if u["email"]==email), None)
    if user and bcrypt.checkpw(password.encode(), user["password"].encode()):
        return jsonify({"message":"Login success","user":user})
    return jsonify({"error":"Invalid credentials"}), 401

@app.get("/health")
def health():
    return jsonify({"status":"ok","message":"Server running"})

print("✅ Flask + MQTT integrated successfully!")


In [ ]:
# ====== Control Door ======
@app.post("/control/door")
def control_door():
    data = request.get_json() or {}
    action = data.get("action")  # open / close
    user_id = data.get("user_id")
    if action not in ["open", "close"]:
        return jsonify({"error":"Invalid action"}), 400

    # نشر الأمر للـ ESP32 عبر MQTT
    mqtt_client.publish("esp32/door/control", action.upper(), qos=1)

    # حفظ العملية في access_logs
    db.access_logs.append({
        "user_id": user_id,
        "action": f"door_{action}",
        "success": True,
        "timestamp": datetime.now().isoformat()
    })

    # تسجيل الحدث في history أيضاً
    db.history.append({
        "event_type": "door",
        "action": action,
        "user_id": user_id,
        "timestamp": datetime.now().isoformat()
    })

    return jsonify({"success": True, "message": f"Door {action} command sent successfully"})


# ====== Control Light ======
@app.post("/control/light")
def control_light():
    data = request.get_json() or {}
    action = data.get("action")  # on / off
    user_id = data.get("user_id")
    device_id = data.get("device_id", "1")  # مثال: رقم الضوء

    if action not in ["on", "off"]:
        return jsonify({"error":"Invalid action"}), 400

    # نشر الأمر للـ ESP32 عبر MQTT
    mqtt_client.publish("esp32/light/control", action.upper(), qos=1)

    # حفظ العملية في access_logs
    db.access_logs.append({
        "user_id": user_id,
        "device_id": device_id,
        "action": f"light_{action}",
        "success": True,
        "timestamp": datetime.now().isoformat()
    })

    # تسجيل الحدث في history أيضاً
    db.history.append({
        "event_type": "light",
        "action": action,
        "user_id": user_id,
        "device_id": device_id,
        "timestamp": datetime.now().isoformat()
    })

    return jsonify({"success": True, "message": f"Light {action} command sent successfully"})


# ====== Access Logs Endpoint ======
@app.get("/access_logs")
def get_access_logs():
    return jsonify({"data": db.access_logs[-50:], "total": len(db.access_logs)})
